In [1]:
! cat Snakefile

OUTDIR = "first_directory"
SNDDIR = "second_directory"

SMP = None

def get_file_names(wildcards):
    ck_output = checkpoints.make_five_files.get(**wildcards).output[0]
    global SMP
    SMP, = glob_wildcards(os.path.join(ck_output, "{sample}.txt"))
    return expand(os.path.join(ck_output, "{SAMPLE}.txt"), SAMPLE=SMP)

def get_second_files(wildcards):
    ck_output = checkpoints.make_five_files.get(**wildcards).output[0]
    SMP2, = glob_wildcards(os.path.join(ck_output, "{sample}.txt"))
    return expand(os.path.join(SNDDIR, "{SM}.tsv"), SM=SMP2)


rule all:
    input: 
        get_second_files,
        expand("list_of_files_{SAMPLE}.txt",SAMPLE=SMP)


checkpoint make_five_files:
    output:
        directory(OUTDIR)
    params:
        o = OUTDIR
    shell:
        """
        mkdir {output};
        for D in $(seq 1 5); do
            touch {params.o}/$RANDOM.txt
        done
        """

rule copy_files:
    input:
        get_file_names
    output:
        os.path.join(SNDDIR, 

In [4]:
!snakemake -c 1 --debug-dag

Assuming unrestricted shared filesystem usage.
Building DAG of jobs...
candidate job all
    wildcards: 
candidate job make_five_files
    wildcards: 
selected job make_five_files
    wildcards: 
candidate job list_all_files
    wildcards: SAMPLE=None
candidate job copy_files
    wildcards: SAMPLE=None
candidate job make_five_files
    wildcards: 
selected job make_five_files
    wildcards: 
selected job copy_files
    wildcards: SAMPLE=None
selected job list_all_files
    wildcards: SAMPLE=None
selected job all
    wildcards: 
candidate job all
    wildcards: 
candidate job make_five_files
    wildcards: 
selected job make_five_files
    wildcards: 
candidate job list_all_files
    wildcards: SAMPLE=None
selected job list_all_files
    wildcards: SAMPLE=None
selected job all
    wildcards: 
candidate job copy_files
    wildcards: SAMPLE=None
candidate job make_five_files
    wildcards: 
selected job make_five_files
    wildcards: 
selected job copy_files
    wildcards: SAMPLE=None
Usi

In [5]:
!ls

first_directory		second_directory  Untitled.ipynb
list_of_files_None.txt	Snakefile


# 2nd edition

In [1]:
! cat Snakefile

OUTDIR = "first_directory"
SNDDIR = "second_directory"

import os


def get_txt_files(wildcards):
    ck_output = checkpoints.make_five_files.get(**wildcards).output[0]
    print([file.split(".")[0] for file in os.listdir(ck_output) if file.endswith('.txt')])
    return [file.split(".")[0] for file in os.listdir(ck_output) if file.endswith('.txt')]


rule all:
    input: 
        OUTDIR,
        expand(f"{OUTDIR}/"+"{SMP}.doc", SMP=get_txt_files)

checkpoint make_five_files:
    output:
        directory(OUTDIR)
    params:
        o = OUTDIR
    shell:
        """
        mkdir {output};
        for D in $(seq 1 5); do
            touch {params.o}/$RANDOM.txt
        done
        """

rule copy_files:
    input:
        "{SMP}.txt"
    output:
        out = "{SMP}.tsv"
    shell:
        """
        touch {output.out}
        """

rule copy2:
    input:
        "{SMP}.tsv"
    output:
        "{SMP}.doc"
    shell:
        """
        touch {output}
        """




In [2]:
! snakemake -c 1 -F

Assuming unrestricted shared filesystem usage.
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                count
---------------  -------
all                    1
make_five_files        1
total                  2

Select jobs to execute...
Execute 1 jobs...

[Thu Aug 22 17:43:06 2024]
localcheckpoint make_five_files:
    output: first_directory
    jobid: 1
    reason: Forced execution
    resources: tmpdir=/tmp
DAG of jobs will be updated after completion.

[Thu Aug 22 17:43:06 2024]
Finished job 1.
1 of 2 steps (50%) done
['10576', '23600', '21071', '24382', '17800']
Select jobs to execute...
Execute 1 jobs...

[Thu Aug 22 17:43:06 2024]
localrule copy_files:
    input: first_directory/10576.txt
    output: first_directory/10576.tsv
    jobid: 5
    reason: Forced execution
    wildcards: SMP=first_directory/10576
    resources: tmpdir=/tmp

[Thu Aug 22 17:43:06

In [3]:
! ls 

first_directory  Snakefile  Untitled.ipynb


In [4]:
! ls first_directory/

10576.doc  17800.doc  21071.doc  23600.doc  24382.doc
10576.tsv  17800.tsv  21071.tsv  23600.tsv  24382.tsv
10576.txt  17800.txt  21071.txt  23600.txt  24382.txt


# 3rd edition

In [10]:
! cat Snakefile

OUTDIR = "first_directory"
SNDDIR = "second_directory"

import os


def get_txt_files(wildcards):
    ck_output = checkpoints.make_five_files.get(**wildcards).output[0]
    print([file.split(".")[0] for file in os.listdir(ck_output) if file.endswith('.txt')])
    return [file.split(".")[0] for file in os.listdir(ck_output) if file.endswith('.txt')]


rule all:
    input: 
        OUTDIR,
        expand(f"{SNDDIR}/"+"{SMP}.doc", SMP=get_txt_files)

checkpoint make_five_files:
    output:
        directory(OUTDIR)
    params:
        o = OUTDIR
    shell:
        """
        mkdir {output};
        for D in $(seq 1 5); do
            touch {params.o}/$RANDOM.txt
        done
        """

rule copy_files:
    input:
        "{SMP}.txt"
    output:
        out = "{SMP}.tsv"
    shell:
        """
        touch {output.out}
        """

rule copy2:
    input:
        "{SMP}.tsv"
    output:
        f"{SNDDIR}/"+"{SMP}.doc"
    shell:
        """
        touch {output}
        """




In [9]:
! snakemake -c 1 -F

Assuming unrestricted shared filesystem usage.
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                count
---------------  -------
all                    1
make_five_files        1
total                  2

Select jobs to execute...
Execute 1 jobs...

[Thu Aug 22 17:46:54 2024]
localcheckpoint make_five_files:
    output: first_directory
    jobid: 1
    reason: Forced execution
    resources: tmpdir=/tmp
DAG of jobs will be updated after completion.

[Thu Aug 22 17:46:54 2024]
Finished job 1.
1 of 2 steps (50%) done
['7907', '16114', '26684', '8109', '32709']
MissingInputException in rule copy_files in file /home/aparemskaya/goldIAV/commonCP/TEST/Snakefile, line 31:
Missing input files for rule copy_files:
    output: 7907.tsv
    wildcards: SMP=7907
    affected files:
        7907.txt


In [11]:
! cat Snakefile

OUTDIR = "first_directory"
SNDDIR = "second_directory"

import os


def get_txt_files(wildcards):
    ck_output = checkpoints.make_five_files.get(**wildcards).output[0]
    print([file.split(".")[0] for file in os.listdir(ck_output) if file.endswith('.txt')])
    return [file.split(".")[0] for file in os.listdir(ck_output) if file.endswith('.txt')]


rule all:
    input: 
        OUTDIR,
        expand(f"{SNDDIR}/"+"{SMP}.doc", SMP=get_txt_files)

checkpoint make_five_files:
    output:
        directory(OUTDIR)
    params:
        o = OUTDIR
    shell:
        """
        mkdir {output};
        for D in $(seq 1 5); do
            touch {params.o}/$RANDOM.txt
        done
        """

rule copy_files:
    input:
        "{SMP}.txt"
    output:
        out = f"{SNDDIR}/"+"{SMP}.tsv"
    shell:
        """
        touch {output.out}
        """

rule copy2:
    input:
        f"{SNDDIR}/"+"{SMP}.tsv"
    output:
        f"{SNDDIR}/"+"{SMP}.doc"
    shell:
        """
        touch {outp

In [12]:
! snakemake -c 1 -F

Assuming unrestricted shared filesystem usage.
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                count
---------------  -------
all                    1
make_five_files        1
total                  2

Select jobs to execute...
Execute 1 jobs...

[Thu Aug 22 17:50:29 2024]
localcheckpoint make_five_files:
    output: first_directory
    jobid: 1
    reason: Forced execution
    resources: tmpdir=/tmp
DAG of jobs will be updated after completion.

[Thu Aug 22 17:50:29 2024]
Finished job 1.
1 of 2 steps (50%) done
['30416', '8841', '8299', '21711', '9997']
MissingInputException in rule copy_files in file /home/aparemskaya/goldIAV/commonCP/TEST/Snakefile, line 31:
Missing input files for rule copy_files:
    output: second_directory/30416.tsv
    wildcards: SMP=30416
    affected files:
        30416.txt


In [13]:
! cat Snakefile

OUTDIR = "first_directory"
SNDDIR = "second_directory"

import os


def get_txt_files(wildcards):
    ck_output = checkpoints.make_five_files.get(**wildcards).output[0]
    print([file.split(".")[0] for file in os.listdir(ck_output) if file.endswith('.txt')])
    return [file.split(".")[0] for file in os.listdir(ck_output) if file.endswith('.txt')]


rule all:
    input: 
        OUTDIR,
        expand(f"{SNDDIR}/"+"{SMP}.doc", SMP=get_txt_files)

checkpoint make_five_files:
    output:
        directory(OUTDIR)
    params:
        o = OUTDIR
    shell:
        """
        mkdir {output};
        for D in $(seq 1 5); do
            touch {params.o}/$RANDOM.txt
        done
        """

rule copy_files:
    input:
        f"{OUTDIR}/"+"{SMP}.txt"
    output:
        out = f"{SNDDIR}/"+"{SMP}.tsv"
    shell:
        """
        touch {output.out}
        """

rule copy2:
    input:
        f"{SNDDIR}/"+"{SMP}.tsv"
    output:
        f"{SNDDIR}/"+"{SMP}.doc"
    shell:
        """
      

In [14]:
! snakemake -c 1 -F

Assuming unrestricted shared filesystem usage.
Building DAG of jobs...
Using shell: /usr/bin/bash
Provided cores: 1 (use --cores to define parallelism)
Rules claiming more threads will be scaled down.
Job stats:
job                count
---------------  -------
all                    1
make_five_files        1
total                  2

Select jobs to execute...
Execute 1 jobs...

[Thu Aug 22 17:52:10 2024]
localcheckpoint make_five_files:
    output: first_directory
    jobid: 1
    reason: Forced execution
    resources: tmpdir=/tmp
DAG of jobs will be updated after completion.

[Thu Aug 22 17:52:11 2024]
Finished job 1.
1 of 2 steps (50%) done
['5114', '11823', '32502', '6262', '21833']
Select jobs to execute...
Execute 1 jobs...

[Thu Aug 22 17:52:11 2024]
localrule copy_files:
    input: first_directory/5114.txt
    output: second_directory/5114.tsv
    jobid: 5
    reason: Forced execution
    wildcards: SMP=5114
    resources: tmpdir=/tmp

[Thu Aug 22 17:52:11 2024]
Finished job 

In [15]:
! ls 

first_directory  second_directory  Snakefile  Untitled.ipynb


In [16]:
! ls  first_directory/

11823.txt  21833.txt  32502.txt  5114.txt  6262.txt


In [17]:
! ls second_directory/

11823.doc  21833.doc  32502.doc  5114.doc  6262.doc
11823.tsv  21833.tsv  32502.tsv  5114.tsv  6262.tsv
